In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import os, glob
from natsort import natsorted
from tensorflow.keras.preprocessing.image import load_img
import cv2
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Load data

In [2]:
data_dir = '../data'
data_dir = pathlib.Path(data_dir)

In [3]:
labels = pd.read_csv('./labels.txt', header = None, names = ['id', 'class', 'x', 'y', 'w', 'h'])

In [4]:
labels_list = labels[['class']].values.tolist()

In [5]:
labels_list[0][0]

0

In [6]:
y_list = []
for x in labels_list:
    #print(x[0])
    y_list.append(x[0])

In [13]:
y_list

[1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,


In [7]:
batch_size = 32
img_height = 64
img_width = 64

In [22]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    
    validation_split=0.2,
    color_mode = 'grayscale',
    shuffle = True,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.


In [23]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    
    color_mode = 'grayscale',
    shuffle = True,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 2000 files belonging to 2 classes.
Using 400 files for validation.


In [24]:
class_names = train_ds.class_names
class_names

['0', '1']

In [148]:
def draw_bbox(img, bbox):

    _, x, y, w, h = bbox
    
    im = img.numpy().astype("uint8")
    
    # Create figure and axes
    fig, ax = plt.subplots()
    # Display the image
    ax.imshow(im)
    # Create a Rectangle patch
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    plt.show()

In [149]:
plt.figure(figsize=(10, 10))
for images in train_ds.take(1):
    for i in range(9):
        draw_bbox(images[i], labels_list[i])

ValueError: not enough values to unpack (expected 5, got 1)

<Figure size 1000x1000 with 0 Axes>

In [25]:
for image_batch, labels_batch in train_ds.take(2):
    print(image_batch.shape)
    print(labels_batch)
    
print(labels.shape)

(32, 64, 64, 1)
tf.Tensor([1 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1], shape=(32,), dtype=int32)
(32, 64, 64, 1)
tf.Tensor([0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0], shape=(32,), dtype=int32)
(2000, 6)


In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [27]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x),y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x),y))

# Training

#### IoU (Intersection over Union) metric 

In [138]:
def iou(true, pred):
    x1, y1, wt, ht = true
    x3, y3, wp, hp = pred

    x2 = x1 + wt
    y2 = y1 + ht

    x4 = x3 + wp
    y4 = y3 + hp    
    
    x_inter1 = max(x1, x3)
    y_inter1 = max(y1, y3)
    x_inter2 = min(x2, x4)
    y_inter2 = min(y2, y4)
    
    w_inter = x_inter2 - x_inter1
    h_inter = y_inter2 - y_inter1
    
    area_inter = w_inter * h_inter
    area_union = wt * ht + wp * hp - area_inter
    
    return area_inter / area_union

# classification model

In [155]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def printScores(model, X, Y):
    print("Classification report:")
    model.predict(X)
    print(classification_report(Y, np.argmax(model.predict(X), axis = 1)))
    print("Confusion matrix:")
    print(confusion_matrix(Y, np.argmax(model.predict(X), axis = 1)))
    print('Wrong predict:')
    print(confusion_matrix(Y, np.argmax(model.predict(X), axis = 1)).sum() - confusion_matrix(Y, np.argmax(model.predict(X), axis = 1)).trace())
    
    
def plotTrainingHistory(model):

    fig, axes= plt.subplots(1,2,figsize=(10,5))
    history = model.history
    axes[0].plot(history['accuracy'])
    axes[0].plot(history['val_accuracy'])
    axes[0].set_ylabel('accuracy')
    axes[0].set_xlabel('epoch')
    axes[0].legend(['train', 'validation'], loc='upper left')

    axes[1].plot(history['loss'])
    axes[1].plot(history['val_loss'])
    axes[1].set_ylabel('loss')
    axes[1].set_xlabel('epoch')
    axes[1].legend(['train', 'validation'], loc='upper left')

In [248]:
def CNN_class(nFilters, kernel_size, pool_size, nNeurons, nHiddenLayers, inputShape, outputWidth):
    inputs = tf.keras.Input(shape=inputShape)
    x = inputs
    for iHidden in range(nHiddenLayers):   
        #x = tf.keras.layers.Rescaling(1./255, input_shape=inputShape)
        x = tf.keras.layers.Conv2D(nFilters, kernel_size=kernel_size)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(x)
        x = tf.keras.layers.Conv2D(nFilters, kernel_size=kernel_size)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(nNeurons, activation=tf.nn.relu)(x)
        x = tf.keras.layers.Dense(int(nNeurons / 2), activation=tf.nn.relu)(x)
        x = tf.keras.layers.Dense(int(nNeurons / 2), activation=tf.nn.relu)(x)

    outputs = tf.keras.layers.Dense(outputWidth, activation=tf.nn.softmax)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [30]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
50/50 [==============================] - 3s 39ms/step - loss: 0.6936 - accuracy: 0.4762 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 2/10
50/50 [==============================] - 2s 36ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 3/10
50/50 [==============================] - 2s 33ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 4/10
50/50 [==============================] - 2s 34ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 5/10
50/50 [==============================] - 2s 36ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 6/10
50/50 [==============================] - 2s 34ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 7/10
50/50 [==============================] - 2s 34ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 8/10
50

In [233]:
%%time 

batch_size = 32
nFilters = 32
kernel_size = 5
pool_size = (5,5)
nNeurons = 128 
nHiddenLayers = 1 
inputShape = (64, 64, 1)
outputWidth = 2

model = CNN_class(nFilters, kernel_size, pool_size, nNeurons, nHiddenLayers, inputShape, outputWidth)

model_fit = model.fit(train_ds, validation_data = val_ds, epochs=5) 
plotTrainingHistory(model_fit)
#printScores(model, X_test, np.argmax(Y_test, axis=1))

Epoch 1/5


ValueError: in user code:

    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "D:\projekty\mgr\custom_nn\mgr-env\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_v

# localization model

### define model

In [112]:
def CNN_localization(nFilters, kernel_size, pool_size, nNeurons, nHiddenLayers, inputShape, outputWidth):
    inputs = tf.keras.Input(shape=inputShape)
    x = inputs  
    x = tf.keras.layers.Conv2D(nFilters, (kernel_size, kernel_size), activation='relu', input_shape=inputShape)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size)(x)
    x = tf.keras.layers.Conv2D(nFilters, (kernel_size, kernel_size), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size)(x)
    x = tf.keras.layers.Conv2D(nFilters, (kernel_size, kernel_size), activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(int(nNeurons), activation='relu')(x)
    x = tf.keras.layers.Dense(int(nNeurons / 2), activation='relu')(x)
    x = tf.keras.layers.Dense(int(nNeurons / 4), activation='relu')(x)
    outputs = tf.keras.layers.Dense(outputWidth)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss = tf.keras.losses.mse, optimizer = 'adam')
    return model

### model training v1 

In [113]:
%%time 

batch_size = 32
nFilters = 32
kernel_size = 3
pool_size = (5,5)
nNeurons = 128 
nHiddenLayers = 1 
inputShape = (320, 320, 3)
outputWidth = 4

model = CNN_localization(nFilters, kernel_size, pool_size, nNeurons, nHiddenLayers, inputShape, outputWidth)
model_fit = model.fit(X_train, Y_train, epochs=30, validation_split = 0.15) 

Epoch 1/30
186/186 [==============================] - 86s 451ms/step - loss: 5414.4102 - val_loss: 2188.9612
Epoch 2/30
186/186 [==============================] - 77s 413ms/step - loss: 1920.0273 - val_loss: 1767.4860
Epoch 3/30
186/186 [==============================] - 76s 409ms/step - loss: 1321.0714 - val_loss: 1023.0104
Epoch 4/30
186/186 [==============================] - 78s 417ms/step - loss: 827.4633 - val_loss: 600.4661
Epoch 5/30
186/186 [==============================] - 76s 409ms/step - loss: 999.0931 - val_loss: 734.2000
Epoch 6/30
186/186 [==============================] - 78s 422ms/step - loss: 572.4796 - val_loss: 541.7265
Epoch 7/30
186/186 [==============================] - 75s 402ms/step - loss: 718.0910 - val_loss: 763.8120
Epoch 8/30
186/186 [==============================] - 74s 398ms/step - loss: 849.2565 - val_loss: 405.2641
Epoch 9/30
186/186 [==============================] - 74s 395ms/step - loss: 415.6583 - val_loss: 294.3798
Epoch 10/30
186/186 [==========

In [123]:
n = 2500
pred = model.predict(X_test[n:n+1])
draw_bbox(X_test[n], Y_test[n], pred[0])

1/1 [==============================] - 0s 37ms/step


In [69]:
pred[0]

array([122.31591 , 118.56896 ,  60.966293,  70.17118 ], dtype=float32)

In [119]:
def draw_bbox(img, true, pred = None):

    xc, yc, w, h = true
    
    start = (int(xc - w / 2), int(yc - h/ 2))
    end = (int(xc + w /2), int(yc + h / 2))
    cv2.rectangle(img, start, end, (0,0,255))
    if type(pred) != None:
        xc, yc, w, h = pred
        start = (int(xc - w / 2), int(yc - h/ 2))
        end = (int(xc + w /2), int(yc + h / 2))
        cv2.rectangle(img, start, end, (0,255,0))
    
    cv2.imshow('bbox', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [100]:
draw_bbox(X_test[1200], Y_test[1200])

### model training v2

In [124]:
%%time 

batch_size = 64
nFilters = 32
kernel_size = 5
pool_size = (3,3)
nNeurons = 128 
nHiddenLayers = 1 
inputShape = (320, 320, 3)
outputWidth = 4

model2 = CNN_localization(nFilters, kernel_size, pool_size, nNeurons, nHiddenLayers, inputShape, outputWidth)
model2_fit = model2.fit(X_train, Y_train, epochs=40, validation_split = 0.15) 

Epoch 1/40
186/186 [==============================] - 171s 908ms/step - loss: 5029.6265 - val_loss: 3043.1282
Epoch 2/40
186/186 [==============================] - 166s 891ms/step - loss: 2398.9636 - val_loss: 2468.6987
Epoch 3/40
186/186 [==============================] - 163s 879ms/step - loss: 1872.7931 - val_loss: 1594.8615
Epoch 4/40
186/186 [==============================] - 162s 869ms/step - loss: 1184.2799 - val_loss: 819.6199
Epoch 5/40
186/186 [==============================] - 164s 884ms/step - loss: 934.6226 - val_loss: 1082.6897
Epoch 6/40
186/186 [==============================] - 164s 883ms/step - loss: 1084.8322 - val_loss: 826.9205
Epoch 7/40
186/186 [==============================] - 164s 882ms/step - loss: 848.3145 - val_loss: 458.0546
Epoch 8/40
186/186 [==============================] - 164s 880ms/step - loss: 587.6010 - val_loss: 552.9735
Epoch 9/40
186/186 [==============================] - 163s 874ms/step - loss: 636.0518 - val_loss: 612.9821
Epoch 10/40
186/186

In [142]:
n = 100
pred = model2.predict(X_test[n:n+1])
draw_bbox(X_test[n], Y_test[n], pred[0])

1/1 [==============================] - 0s 50ms/step


In [150]:
iou_all = []
errors = []
for i in range(Y_test.shape[0]):
    pred = model2.predict(X_test[i:i+1], verbose = 0)
    result = iou(Y_test[i], pred[0])
    if result <= 1:
        iou_all.append(result)
    else:
        errors.append([i, result])

In [156]:
print('Number of IoU < 1: ', len(iou_all))
print('Mean IoU: ', np.mean(iou_all))

Number of IoU < 1:  2894
Mean IoU:  0.6193087811257244
